# Image Visualization



In [1]:
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=KR4DCNiU0KR1z1mwDWjl6pGUcC-7X6h416hwqTGfg5Y&tc=WfEW_VdKzGS-syRBeLBQMmtPJY7nSmKk0qe67hvyZBk&cc=wN6LAB49V1_N1up88sJwXTak7jl7AtHygWmW0hI1EXw

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AVHEtk5yfr2weg15nvFUQYFd5wiY8GTtM0yGtrLhieR3F-hD4vNW8aMWNoM

Successfully saved authorization token.


In [2]:
import folium


def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
  ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

In [7]:
# Load an image.
image = ee.Image('LANDSAT/LC08/C02/T1_TOA/LC08_044034_20140318')

# Define the visualization parameters.
image_viz_params = {
    'bands': ['B5', 'B4', 'B3'],
    'min': 0,
    'max': 0.5,
    'gamma': [0.95, 1.1, 1]
}

# Define a map centered on San Francisco Bay.
map_l8 = folium.Map(location=[37.5010, -122.1899], zoom_start=10)

# Add the image layer to the map and display it.
map_l8.add_ee_layer(image, image_viz_params, 'false color composite')
display(map_l8)

In this example, band `'B5'` is assigned to red, `'B4'` is assigned to green, and `'B3'` is assigned to blue.

## Color palettes

To display a single band of an image in color, set the `palette` parameter with a color ramp represented by a list of CSS-style color strings. (See [this reference](http://en.wikipedia.org/wiki/Web_colors) for more information). The following example illustrates how to use colors from cyan (`'00FFFF'`) to blue (`'0000FF'`) to render a [ Normalized Difference Water Index (NDWI)](http://www.tandfonline.com/doi/abs/10.1080/01431169608948714) image:

In [6]:
# Load an image.
image = ee.Image('LANDSAT/LC08/C02/T1_TOA/LC08_044034_20140318')

# Create an NDWI image, define visualization parameters and display.
ndwi = image.normalizedDifference(['B3', 'B5'])
ndwi_viz = {'min': 0.5, 'max': 1, 'palette': ['00FFFF', '0000FF']}

# Define a map centered on San Francisco Bay.
map_ndwi = folium.Map(location=[37.5010, -122.1899], zoom_start=10)

# Add the image layer to the map and display it.
map_ndwi.add_ee_layer(ndwi, ndwi_viz, 'NDWI')
display(map_ndwi)

In [8]:
# Mask the non-watery parts of the image, where NDWI < 0.4.
ndwi_masked = ndwi.updateMask(ndwi.gte(0.4))

# Define a map centered on San Francisco Bay.
map_ndwi_masked = folium.Map(location=[37.5010, -122.1899], zoom_start=10)

# Add the image layer to the map and display it.
map_ndwi_masked.add_ee_layer(ndwi_masked, ndwi_viz, 'NDWI masked')
display(map_ndwi_masked)

In [9]:
image_rgb = image.visualize(**{'bands': ['B5', 'B4', 'B3'], 'max': 0.5})
ndwi_rgb = ndwi_masked.visualize(**{
    'min': 0.5,
    'max': 1,
    'palette': ['00FFFF', '0000FF']
})

In [10]:
# Mosaic the visualization layers and display (or export).
mosaic = ee.ImageCollection([image_rgb, ndwi_rgb]).mosaic()

# Define a map centered on San Francisco Bay.
map_mosaic = folium.Map(location=[37.5010, -122.1899], zoom_start=10)

# Add the image layer to the map and display it.
map_mosaic.add_ee_layer(mosaic, None, 'mosaic')
display(map_mosaic)

In [11]:
# Create a circle by drawing a 20000 meter buffer around a point.
roi = ee.Geometry.Point([-122.4481, 37.7599]).buffer(20000)
mosaic_clipped = mosaic.clip(roi)

# Define a map centered on San Francisco.
map_mosaic_clipped = folium.Map(location=[37.7599, -122.4481], zoom_start=10)

# Add the image layer to the map and display it.
map_mosaic_clipped.add_ee_layer(mosaic_clipped, None, 'mosaic clipped')
display(map_mosaic_clipped)

In [12]:
# Load 2012 MODIS land cover and select the IGBP classification.
cover = ee.Image('MODIS/051/MCD12Q1/2012_01_01').select('Land_Cover_Type_1')

# Define a palette for the 18 distinct land cover classes.
igbp_palette = [
    'aec3d4',  # water
    '152106',
    '225129',
    '369b47',
    '30eb5b',
    '387242',  # forest
    '6a2325',
    'c3aa69',
    'b76031',
    'd9903d',
    '91af40',  # shrub, grass
    '111149',  # wetlands
    'cdb33b',  # croplands
    'cc0013',  # urban
    '33280d',  # crop mosaic
    'd7cdcc',  # snow and ice
    'f7e084',  # barren
    '6f6f6f'   # tundra
]

# Define a map centered on the United States.
map_palette = folium.Map(location=[40.413, -99.229], zoom_start=5)

# Add the image layer to the map and display it. Specify the min and max labels
# and the color palette matching the labels.
map_palette.add_ee_layer(
    cover, {'min': 0, 'max': 17, 'palette': igbp_palette}, 'IGBP classes')
display(map_palette)

In [13]:
cover = ee.Image('MODIS/051/MCD12Q1/2012_01_01').select('Land_Cover_Type_1')

# Define an SLD style of discrete intervals to apply to the image.
sld_intervals = """
<RasterSymbolizer>
  <ColorMap  type="intervals" extended="false" >
    <ColorMapEntry color="#aec3d4" quantity="0" label="Water"/>
    <ColorMapEntry color="#152106" quantity="1" label="Evergreen Needleleaf Forest"/>
    <ColorMapEntry color="#225129" quantity="2" label="Evergreen Broadleaf Forest"/>
    <ColorMapEntry color="#369b47" quantity="3" label="Deciduous Needleleaf Forest"/>
    <ColorMapEntry color="#30eb5b" quantity="4" label="Deciduous Broadleaf Forest"/>
    <ColorMapEntry color="#387242" quantity="5" label="Mixed Deciduous Forest"/>
    <ColorMapEntry color="#6a2325" quantity="6" label="Closed Shrubland"/>
    <ColorMapEntry color="#c3aa69" quantity="7" label="Open Shrubland"/>
    <ColorMapEntry color="#b76031" quantity="8" label="Woody Savanna"/>
    <ColorMapEntry color="#d9903d" quantity="9" label="Savanna"/>
    <ColorMapEntry color="#91af40" quantity="10" label="Grassland"/>
    <ColorMapEntry color="#111149" quantity="11" label="Permanent Wetland"/>
    <ColorMapEntry color="#cdb33b" quantity="12" label="Cropland"/>
    <ColorMapEntry color="#cc0013" quantity="13" label="Urban"/>
    <ColorMapEntry color="#33280d" quantity="14" label="Crop, Natural Veg. Mosaic"/>
    <ColorMapEntry color="#d7cdcc" quantity="15" label="Permanent Snow, Ice"/>
    <ColorMapEntry color="#f7e084" quantity="16" label="Barren, Desert"/>
    <ColorMapEntry color="#6f6f6f" quantity="17" label="Tundra"/>
  </ColorMap>
</RasterSymbolizer>"""

# Apply the SLD style to the image.
cover_sld = cover.sldStyle(sld_intervals)

# Define a map centered on the United States.
map_sld_categorical = folium.Map(location=[40.413, -99.229], zoom_start=5)

# Add the image layer to the map and display it.
map_sld_categorical.add_ee_layer(cover_sld, None, 'IGBP classes styled')
display(map_sld_categorical)

In [14]:
# Load SRTM Digital Elevation Model data.
image = ee.Image('CGIAR/SRTM90_V4')

# Define an SLD style of discrete intervals to apply to the image.
sld_intervals = """
    <RasterSymbolizer>
      <ColorMap type="intervals" extended="false" >
        <ColorMapEntry color="#0000ff" quantity="0" label="0"/>
        <ColorMapEntry color="#00ff00" quantity="100" label="1-100" />
        <ColorMapEntry color="#007f30" quantity="200" label="110-200" />
        <ColorMapEntry color="#30b855" quantity="300" label="210-300" />
        <ColorMapEntry color="#ff0000" quantity="400" label="310-400" />
        <ColorMapEntry color="#ffff00" quantity="1000" label="410-1000" />
      </ColorMap>
    </RasterSymbolizer>"""

# Define an sld style color ramp to apply to the image.
sld_ramp = """
    <RasterSymbolizer>
      <ColorMap type="ramp" extended="false" >
        <ColorMapEntry color="#0000ff" quantity="0" label="0"/>
        <ColorMapEntry color="#00ff00" quantity="100" label="100" />
        <ColorMapEntry color="#007f30" quantity="200" label="200" />
        <ColorMapEntry color="#30b855" quantity="300" label="300" />
        <ColorMapEntry color="#ff0000" quantity="400" label="400" />
        <ColorMapEntry color="#ffff00" quantity="500" label="500" />
      </ColorMap>
    </RasterSymbolizer>"""

# Define a map centered on the United States.
map_sld_interval = folium.Map(location=[40.413, -99.229], zoom_start=5)

# Add the image layers to the map and display it.
map_sld_interval.add_ee_layer(
    image.sldStyle(sld_intervals), None, 'SLD intervals')
map_sld_interval.add_ee_layer(image.sldStyle(sld_ramp), None, 'SLD ramp')
display(map_sld_interval.add_child(folium.LayerControl()))

In [15]:
# Load a Landsat 8 raw image.
image = ee.Image('LANDSAT/LC08/C02/T1/LC08_044034_20140318')

# Define a RasterSymbolizer element with '_enhance_' for a placeholder.
template_sld = """
<RasterSymbolizer>
  <ContrastEnhancement><_enhance_/></ContrastEnhancement>
  <ChannelSelection>
    <RedChannel>
      <SourceChannelName>B5</SourceChannelName>
    </RedChannel>
    <GreenChannel>
      <SourceChannelName>B4</SourceChannelName>
    </GreenChannel>
    <BlueChannel>
      <SourceChannelName>B3</SourceChannelName>
    </BlueChannel>
  </ChannelSelection>
</RasterSymbolizer>"""

# Get SLDs with different enhancements.
equalize_sld = template_sld.replace('_enhance_', 'Histogram')
normalize_sld = template_sld.replace('_enhance_', 'Normalize')

# Define a map centered on San Francisco Bay.
map_sld_continuous = folium.Map(location=[37.5010, -122.1899], zoom_start=10)

# Add the image layers to the map and display it.
map_sld_continuous.add_ee_layer(
    image, {'bands': ['B5', 'B4', 'B3'], 'min': 0, 'max': 15000}, 'Linear')
map_sld_continuous.add_ee_layer(
    image.sldStyle(equalize_sld), None, 'Equalized')
map_sld_continuous.add_ee_layer(
    image.sldStyle(normalize_sld), None, 'Normalized')
display(map_sld_continuous.add_child(folium.LayerControl()))

In [16]:
# Fetch a digital elevation model.
image = ee.Image('CGIAR/SRTM90_V4')

# Request a default thumbnail of the DEM with defined linear stretch.
# Set masked pixels (ocean) to 1000 so they map as gray.
thumbnail_1 = image.unmask(1000).getThumbURL({
    'min': 0,
    'max': 3000,
    'dimensions': 500,
})
print('Default extent:', thumbnail_1)

# Specify region by rectangle, define palette, set larger aspect dimension size.
thumbnail_2 = image.getThumbURL({
    'min': 0,
    'max': 3000,
    'palette': [
        '00A600', '63C600', 'E6E600', 'E9BD3A', 'ECB176', 'EFC2B3', 'F2F2F2'
    ],
    'dimensions': 500,
    'region': ee.Geometry.Rectangle([-84.6, -55.9, -32.9, 15.7]),
})
print('Rectangle region and palette:', thumbnail_2)

# Specify region by a linear ring and set display CRS as Web Mercator.
thumbnail_3 = image.getThumbURL({
    'min': 0,
    'max': 3000,
    'palette': [
        '00A600', '63C600', 'E6E600', 'E9BD3A', 'ECB176', 'EFC2B3', 'F2F2F2'
    ],
    'region':
        ee.Geometry.LinearRing([[-84.6, 15.7], [-84.6, -55.9], [-32.9, -55.9]]),
    'dimensions': 500,
    'crs': 'EPSG:3857'
})
print('Linear ring region and specified crs:', thumbnail_3)

Default extent: https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/3bbe407c4fb4d95e5a03d51383de3238-6c889d5900bc70626d6db14bf0d853e1:getPixels
Rectangle region and palette: https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/c422a98c58645c378ee7513b126ef150-f787017b211ec920875462e2c0210323:getPixels
Linear ring region and specified crs: https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/05c351b5fe4884b734bd9a27b912a076-6d3fcb86613cd0facd369e09ca611269:getPixels
